In [1]:
"Book : Design, Testing, and Optimization of Trading Systems"

'Book : Design, Testing, and Optimization of Trading Systems'

In [2]:
viewdict = {
            "AMZN": 0.10,
            "BAC": 0.30,
            "COST": 0.05,
            "DIS": 0.05,
            "DPZ": 0.20,
            "KO": -0.05,  # 5% down
            "MCD": 0.15,
            "MSFT": 0.10,
            "NAT": 0.50, 
            "SBUX": 0.10
        }

Using RSI and MA backtest on last 3 months of data and get the best parameter which gives best return.
Take the number of stocks on positive return after sorting in descending order
Those number of stocks will become the  part of the portfolio in the BL model

For evaluating the view, take the dataframe with signals of buy and sell i.e entries and exits for each stock with the backtested values and then attach the sentiments on those dates for a particular stock and give a view for that stock.

In [10]:
import numpy as np
import scipy.stats as stats
import vectorbt as vbt
import warnings
warnings.filterwarnings('ignore')


/home/diti/miniconda3/envs/devbox/lib/python3.12/site-packages/telegram/utils/request.py:49: UserWarning: python-telegram-bot is using upstream urllib3. This is allowed but not supported by python-telegram-bot maintainers.
  warnings.warn(


In [49]:
start = "2016-01-01 UTC"
end = "2020-01-01 UTC"
prices = vbt.YFData.download("AAPL", start=start, end=end).get("Close")

In [50]:
d = prices.vbt.rolling_split(
    n=30,
    window_len=365*2,
    set_lens=(180,),
    left_to_right=False,
    plot=True
)

In [51]:
d

FigureWidget({
    'data': [{'colorscale': [[0.0, '#1f77b4'], [1.0, '#1f77b4']],
              'hoverongaps': False,
              'name': '0',
              'showlegend': True,
              'showscale': False,
              'type': 'heatmap',
              'uid': '23fa2dd8-9fcf-4aeb-aa28-94363483bba0',
              'x': array([datetime.datetime(2016, 1, 4, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2016, 1, 5, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2016, 1, 6, 5, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2019, 12, 27, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2019, 12, 30, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2019, 12, 31, 5, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15

In [30]:
(in_price, in_indexes), (out_price, out_indexes) = prices.vbt.rolling_split(
    n=30,
    window_len=365*2,
    set_lens=(180,),
    left_to_right=False,
)

In [52]:
out_price

split_idx,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,42.971344,40.854931,40.211735,39.074306,43.470295,44.226631,45.159542,44.827202,44.034340,45.418285,...,40.310261,37.570805,36.648087,36.502266,41.651314,41.066509,41.897163,45.171734,46.578548,47.834114
1,42.557522,39.807377,40.968445,38.530426,43.784801,44.539959,45.536968,44.803486,43.941761,45.318581,...,40.422619,37.326973,36.765217,37.711834,40.862457,41.525055,41.412228,46.835419,46.897839,48.765598
2,42.195713,39.367531,40.776894,38.698330,43.995258,44.428398,45.888294,44.079453,44.433147,45.446770,...,40.864834,37.346092,36.404259,37.362831,40.910477,41.827545,41.510647,45.865540,46.979462,48.940845
3,42.245380,39.674957,41.178905,38.833103,44.305046,44.713261,46.047340,44.271736,43.659286,45.197525,...,39.557266,37.707058,35.856842,36.975571,40.675209,41.851551,42.948666,45.310978,47.293957,49.101696
4,42.096409,39.414825,41.318420,38.383820,44.938774,44.663399,45.923916,44.024857,44.010601,45.548847,...,39.189152,37.750088,36.590714,39.502277,41.025707,41.981190,43.431206,44.842834,48.038177,49.809895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,46.262501,44.180412,39.189152,37.750088,36.590714,39.502277,41.025707,41.981190,43.431206,44.842834,...,51.396416,51.589943,53.452663,53.418793,56.696674,58.847267,63.561260,64.126625,65.143250,68.974510
176,44.429020,42.236969,39.698303,33.989895,37.037724,39.786758,40.855251,41.568268,43.623272,45.246151,...,49.020844,51.810081,52.671280,54.916214,56.916821,60.177784,64.170273,64.987961,65.698906,70.342972
177,42.306305,41.766041,38.460049,35.440903,37.257648,39.805882,41.004101,42.005192,44.108219,45.306164,...,49.952206,52.422119,52.910767,54.928322,57.190174,61.885681,63.726246,64.844818,65.866318,70.316284
178,42.258488,40.276802,37.489525,35.362015,37.487141,40.936558,40.912876,42.216457,44.681980,45.601456,...,49.388546,54.088879,52.659187,54.284821,58.182018,62.292088,64.483299,64.095070,66.761658,70.733604


In [32]:
def simulate_all_params(price, windows, **kwargs):
    fast_ma, slow_ma = vbt.MA.run_combs(
        price,
        windows,
        r=2,
        short_names=["fast","slow"]
    )
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)

    pf = vbt.Portfolio.from_signals(price, entries, exits, **kwargs)
    return pf.sharpe_ratio()

In [53]:
def simulate_best_params(price, best_fast_windows, best_slow_windows, **kwargs):
    fast_ma = vbt.MA.run(
        price,
        window=best_fast_windows,
        per_column=True
    )
    slow_ma=vbt.MA.run(
        price,
        window=best_slow_windows,
        per_column=True
    )

    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)

    pf = vbt.Portfolio.from_signals(price, entries, exits, **kwargs)
    return pf.sharpe_ratio()

In [54]:
def get_best_index(performance):
    return performance[performance.groupby("split_idx").idxmax()].index

In [55]:
def get_best_params(best_index, level_name):
    return best_index.get_level_values(level_name).to_numpy()

In [44]:
in_sharpe = simulate_all_params(
    in_price,
    np.arange(10, 40),
    direction="both",
    freq="d"
)

In [45]:
in_sharpe.vbt.plot()

FigureWidget({
    'data': [{'name': 'sharpe_ratio',
              'showlegend': True,
              'type': 'scatter',
              'uid': '531a5027-3d6d-44b4-9586-63b193440f7b',
              'x': [(10, 11, 0), (10, 11, 1), (10, 11, 2), ..., (38, 39, 27), (38,
                    39, 28), (38, 39, 29)],
              'y': array([ 1.48299174,  1.31566665,  1.3825952 , ..., -0.87533486, -0.80065718,
                          -0.6953146 ])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'width': 700}
})

In [56]:
in_best_index = get_best_index(in_sharpe)

in_best_fast_windows = get_best_params(
    in_best_index,
    "fast_window"
)

in_best_slow_windows = get_best_params(
    in_best_index,
    "slow_window"
)

in_best_window_pairs = np.array(
    list(
        zip(
            in_best_fast_windows,
            in_best_slow_windows
        )
    )
)

out_test_sharpe = simulate_best_params(
    out_price,
    in_best_fast_windows,
    in_best_slow_windows,
    direction="both",
    freq="d"
)

In [59]:
in_best_fast_windows

array([10, 12, 12, 10, 12, 10, 10, 18, 18, 18, 18, 18, 18, 18, 18, 18, 23,
       23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24])

In [60]:
in_best_slow_windows

array([11, 13, 13, 11, 13, 11, 11, 23, 23, 23, 23, 23, 23, 23, 23, 23, 26,
       26, 26, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25])

In [57]:
out_test_sharpe

ma_window  ma_window  split_idx
10         11         0            0.104949
12         13         1            0.318319
                      2            0.971218
10         11         3            1.386787
12         13         4            1.303270
10         11         5            2.133303
                      6            2.043532
18         23         7            1.756911
                      8            2.219373
                      9            2.283886
                      10           2.544002
                      11           2.724652
                      12           2.389906
                      13           2.838683
                      14           2.393309
                      15           1.116314
23         26         16           0.670550
                      17           0.594140
                      18           0.816456
24         25         19           0.276336
                      20          -0.052512
                      21          -0.363497


In [58]:
t, p = stats.ttest_ind(
    a=out_test_sharpe.values,
    b=in_sharpe[in_best_index].values,
    alternative="greater"
)

t,p

(-1.084925756504446, 0.8587781936832611)

In [66]:
in_sharpe[in_best_index].values

array([1.48299174, 1.65829639, 1.65773656, 1.26682913, 0.84855086,
       0.74689375, 0.7638398 , 0.75910151, 0.65572604, 0.72939225,
       0.80372315, 1.09832068, 0.99308825, 1.14479249, 1.01930191,
       1.04498651, 1.3131161 , 1.37596945, 1.43877296, 1.81287839,
       1.8389251 , 2.14768585, 2.0663446 , 1.40120782, 1.42426319,
       1.49925815, 1.5477486 , 1.61025142, 1.65815904, 1.75581214])

In [70]:
out_test_sharpe

ma_window  ma_window  split_idx
10         11         0            0.104949
12         13         1            0.318319
                      2            0.971218
10         11         3            1.386787
12         13         4            1.303270
10         11         5            2.133303
                      6            2.043532
18         23         7            1.756911
                      8            2.219373
                      9            2.283886
                      10           2.544002
                      11           2.724652
                      12           2.389906
                      13           2.838683
                      14           2.393309
                      15           1.116314
23         26         16           0.670550
                      17           0.594140
                      18           0.816456
24         25         19           0.276336
                      20          -0.052512
                      21          -0.363497


In [17]:

rvs = st.norm.rvs(loc=5, scale=10, size=(50, 2))
print(rvs)
print(st.ttest_1samp(rvs, 5.0))

[[-16.71094331   7.98706688]
 [  2.99066332  11.28806518]
 [ -8.51241338  12.43989023]
 [ 25.01124231  -9.47642126]
 [  8.49202017  13.68308096]
 [-10.62477625  -6.32113038]
 [ -2.53288771   9.72161092]
 [ 12.33239514  -2.82870138]
 [-14.37597564 -10.39329065]
 [ -0.81072806   5.64079876]
 [ 13.18991782   5.59385488]
 [ -7.10168565  21.94305699]
 [ -2.89718137   8.9054597 ]
 [ -6.98769842  31.28575332]
 [ 16.24447992  -5.35081847]
 [ -8.88767293 -11.3170015 ]
 [ 22.1048468    4.99283225]
 [ 19.81774257   8.28133491]
 [ -4.7859835   -4.90434393]
 [  7.56114187  11.02547098]
 [  5.85626015   9.33538195]
 [ 19.60072096   4.03652816]
 [ 24.69813495   2.12152611]
 [  0.64806246   7.71439413]
 [  6.82154903  15.48014775]
 [ 20.7875683   18.6022884 ]
 [ 17.14181105  22.77557028]
 [ 10.59772535   2.2481689 ]
 [ -5.71634821  23.12896933]
 [  8.87059265  11.21472552]
 [  6.43548296  10.25344331]
 [ 10.47445561  -4.75801397]
 [-21.4138727    7.77607714]
 [ -7.90255103  -5.84294478]
 [  3.85695225

In [18]:
np.random.randint(0,2,100)

array([1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1])